In [9]:
pip install numpy scipy mne tqdm



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os  # ✅ Import os module

# ✅ Define the path to your EEG dataset
edf_dir = r"C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev"  # ✅ Set dataset directory

# ✅ Scan for .edf files
edf_files = []
for root, _, files in os.walk(edf_dir):  # Recursively search for `.edf`
    for file in files:
        if file.endswith('.edf'):
            edf_files.append(os.path.join(root, file))

# ✅ Check if files are found
if edf_files:
    print(f"✅ Found {len(edf_files)} EDF files in subdirectories")
    print("Example files:", edf_files[:5])  # Show first 5
else:
    print("❌ No EDF files found! Check the folder path.")


✅ Found 438 EDF files in subdirectories
Example files: ['C:\\Users\\alsaf\\Downloads\\Capstone2\\data\\data\\v2.0.3\\edf\\dev\\aaaaaajy\\s001_2003\\02_tcp_le\\aaaaaajy_s001_t000.edf', 'C:\\Users\\alsaf\\Downloads\\Capstone2\\data\\data\\v2.0.3\\edf\\dev\\aaaaaajy\\s002_2003\\01_tcp_ar\\aaaaaajy_s002_t000.edf', 'C:\\Users\\alsaf\\Downloads\\Capstone2\\data\\data\\v2.0.3\\edf\\dev\\aaaaaajy\\s002_2003\\01_tcp_ar\\aaaaaajy_s002_t001.edf', 'C:\\Users\\alsaf\\Downloads\\Capstone2\\data\\data\\v2.0.3\\edf\\dev\\aaaaaajy\\s002_2003\\01_tcp_ar\\aaaaaajy_s002_t002.edf', 'C:\\Users\\alsaf\\Downloads\\Capstone2\\data\\data\\v2.0.3\\edf\\dev\\aaaaaajy\\s003_2003\\01_tcp_ar\\aaaaaajy_s003_t000.edf']


In [11]:
for edf_file in edf_files:
    edf_path = os.path.join(edf_dir, edf_file)
    print(f"🔍 Checking: {edf_path}")

    if not os.path.exists(edf_path):
        print(f"❌ File not found: {edf_path}")
    else:
        print(f"✅ File exists: {edf_path}")


🔍 Checking: C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev\aaaaaajy\s001_2003\02_tcp_le\aaaaaajy_s001_t000.edf
✅ File exists: C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev\aaaaaajy\s001_2003\02_tcp_le\aaaaaajy_s001_t000.edf
🔍 Checking: C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev\aaaaaajy\s002_2003\01_tcp_ar\aaaaaajy_s002_t000.edf
✅ File exists: C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev\aaaaaajy\s002_2003\01_tcp_ar\aaaaaajy_s002_t000.edf
🔍 Checking: C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev\aaaaaajy\s002_2003\01_tcp_ar\aaaaaajy_s002_t001.edf
✅ File exists: C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev\aaaaaajy\s002_2003\01_tcp_ar\aaaaaajy_s002_t001.edf
🔍 Checking: C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev\aaaaaajy\s002_2003\01_tcp_ar\aaaaaajy_s002_t002.edf
✅ File exists: C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev\aaaaaajy\s002_2003\01_tcp_ar\aaaaaajy_s002_t002.

In [12]:
import os
import numpy as np
import mne
import scipy.signal as signal
from tqdm import tqdm

# ✅ Paths
edf_dir = r"C:\Users\alsaf\Downloads\Capstone2\data\data\v2.0.3\edf\dev"  # Raw EEG files (includes subfolders)
output_dir = r"C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed"  # Processed files
os.makedirs(output_dir, exist_ok=True)

# ✅ Filtering settings
lowcut = 0.5  # Lower cutoff for bandpass filter
highcut = 70  # Upper cutoff for bandpass filter
fs = 256  # EEG Sampling rate (Adjust if different)
notch_freq = 50  # Powerline noise frequency
order = 4  # Filter order
min_channels = 16  # Minimum channels required (Update as needed)

# ✅ Bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data, axis=1)

# ✅ Notch filter function (Removes 50Hz noise)
def notch_filter(data, notch_freq, fs):
    Q = 30.0
    b, a = signal.iirnotch(notch_freq / (fs / 2), Q)
    return signal.filtfilt(b, a, data, axis=1)

# ✅ Z-score normalization
def normalize(data):
    mean = np.mean(data, axis=1, keepdims=True)
    std = np.std(data, axis=1, keepdims=True)
    return (data - mean) / (std + 1e-8)  # Avoid division by zero

# ✅ Scan all `.edf` files from subdirectories
edf_files = []
for root, _, files in os.walk(edf_dir):  # Recursively search for `.edf`
    for file in files:
        if file.endswith('.edf'):
            edf_files.append(os.path.join(root, file))

print(f"✅ Found {len(edf_files)} EDF files for processing...\n")

# ✅ Process each `.edf` file
for edf_path in tqdm(edf_files, desc="Processing EEG Files"):
    try:
        # Load EEG data
        raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
        raw.resample(fs)  # Resample to standard sampling rate
        data = raw.get_data()  # Shape: (n_channels, n_samples)

        # ✅ Ensure minimum required channels
        if data.shape[0] < min_channels:
            print(f"⚠️ Skipping {edf_path} (Only {data.shape[0]} channels found)")
            continue

        # Apply filters
        data = bandpass_filter(data, lowcut, highcut, fs, order)
        data = notch_filter(data, notch_freq, fs)

        # Normalize
        data = normalize(data)

        # Save as `.npy` (use filename only, not full path)
        output_filename = os.path.basename(edf_path).replace(".edf", ".npy")
        output_path = os.path.join(output_dir, output_filename)
        np.save(output_path, data)

        print(f"✅ Saved: {output_path} | Shape: {data.shape}")

    except Exception as e:
        print(f"❌ Error processing {edf_path}: {e}")

print("\n✅ All EEG files have been processed & saved!")


✅ Found 438 EDF files for processing...



Processing EEG Files:   0%|          | 1/438 [00:04<36:12,  4.97s/it]

✅ Saved: C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed\aaaaaajy_s001_t000.npy | Shape: (33, 448000)
✅ Saved: C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed\aaaaaajy_s002_t000.npy | Shape: (32, 5120)
✅ Saved: C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed\aaaaaajy_s002_t001.npy | Shape: (32, 1792)


Processing EEG Files:   1%|          | 4/438 [00:05<07:33,  1.04s/it]

✅ Saved: C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed\aaaaaajy_s002_t002.npy | Shape: (32, 69632)


...

✅ Saved: C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed\aaaaahie_s031_t014.npy | Shape: (20, 921856)
Sampling frequency of the instance is already 256.0, returning unmodified.


Processing EEG Files: 100%|█████████▉| 437/438 [06:08<00:01,  1.90s/it]

✅ Saved: C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed\aaaaahie_s031_t015.npy | Shape: (20, 921856)
Sampling frequency of the instance is already 256.0, returning unmodified.


Processing EEG Files: 100%|██████████| 438/438 [06:11<00:00,  1.18it/s]

✅ Saved: C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed\aaaaahie_s031_t016.npy | Shape: (20, 921856)

✅ All EEG files have been processed & saved!


In [13]:
import numpy as np
import os

output_dir = r"C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed"

# ✅ Load and check a sample
sample_file = os.listdir(output_dir)[0]  # Pick first file
sample_data = np.load(os.path.join(output_dir, sample_file))

print(f"✅ Sample EEG Shape: {sample_data.shape}")  # Should be (n_channels, n_samples)
print(f"✅ Mean: {np.mean(sample_data):.4f}, Std Dev: {np.std(sample_data):.4f}")  # Should be ~0 (normalized)


✅ Sample EEG Shape: (33, 448000)
✅ Mean: -0.0000, Std Dev: 0.9994


TS -TCC

In [14]:
pip install torch torchvision torchaudio numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# ✅ Paths
npy_dir = r"C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed"

# ✅ Augmentation Function: Adds noise + randomly crops EEG signals
def augment_eeg(eeg_data, crop_size=10000, noise_level=0.05):
    """Apply random cropping & noise addition to EEG data."""
    total_samples = eeg_data.shape[1]

    # ✅ Random Crop
    start_idx = np.random.randint(0, total_samples - crop_size)
    eeg_cropped = eeg_data[:, start_idx:start_idx + crop_size]

    # ✅ Add Gaussian Noise
    noise = noise_level * np.random.randn(*eeg_cropped.shape)
    eeg_noisy = eeg_cropped + noise

    return eeg_noisy

# ✅ Define Custom EEG Dataset
class EEGDataset(Dataset):
    def __init__(self, npy_dir, crop_size=10000):
        self.npy_files = [os.path.join(npy_dir, f) for f in os.listdir(npy_dir) if f.endswith('.npy')]
        self.crop_size = crop_size

    def __len__(self):
        return len(self.npy_files)

    def __getitem__(self, idx):
        # ✅ Load EEG segment
        eeg_data = np.load(self.npy_files[idx])  # Shape: (n_channels, n_samples)
        eeg_data = torch.tensor(eeg_data, dtype=torch.float32)

        # ✅ Generate two different augmentations of the same EEG signal
        eeg_1 = augment_eeg(eeg_data.numpy(), crop_size=self.crop_size)
        eeg_2 = augment_eeg(eeg_data.numpy(), crop_size=self.crop_size)

        return torch.tensor(eeg_1, dtype=torch.float32), torch.tensor(eeg_2, dtype=torch.float32)

# ✅ Create DataLoader
batch_size = 32
train_dataset = EEGDataset(npy_dir, crop_size=10000)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

print(f"✅ Loaded {len(train_dataset)} EEG segments for TS-TCC training.")


✅ Loaded 438 EEG segments for TS-TCC training.


In [16]:
import os
import numpy as np

npy_dir = r"C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed"

# ✅ List all `.npy` files
npy_files = [f for f in os.listdir(npy_dir) if f.endswith('.npy')]

if npy_files:
    print(f"✅ Found {len(npy_files)} .npy files")
    sample_file = os.path.join(npy_dir, npy_files[0])
    sample_data = np.load(sample_file)
    print(f"✅ Sample File: {sample_file}, Shape: {sample_data.shape}")
else:
    print("❌ No .npy files found! Check preprocessing step.")


✅ Found 438 .npy files
✅ Sample File: C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed\aaaaaajy_s001_t000.npy, Shape: (33, 448000)


In [17]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)


In [18]:
import numpy as np
import os

npy_dir = r"C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed"

# ✅ Check smallest EEG segment
min_length = float("inf")

for file in os.listdir(npy_dir):
    if file.endswith(".npy"):
        eeg_data = np.load(os.path.join(npy_dir, file))
        min_length = min(min_length, eeg_data.shape[1])

print(f"✅ Smallest EEG segment length: {min_length}")


✅ Smallest EEG segment length: 256


In [19]:
import numpy as np

def augment_eeg(eeg_data, crop_size=10000, noise_level=0.05):
    """Apply random cropping & noise addition to EEG data."""
    total_samples = eeg_data.shape[1]

    # ✅ If EEG is shorter than crop size, use full length
    if total_samples < crop_size:
        print(f"⚠️ Warning: EEG sample is too short ({total_samples} samples). Using full length.")
        crop_size = total_samples  # Adjust crop size

    # ✅ Random Crop
    start_idx = np.random.randint(0, total_samples - crop_size + 1)
    eeg_cropped = eeg_data[:, start_idx:start_idx + crop_size]

    # ✅ Add Gaussian Noise
    noise = noise_level * np.random.randn(*eeg_cropped.shape)
    eeg_noisy = eeg_cropped + noise

    return eeg_noisy


In [20]:
train_dataset = EEGDataset(npy_dir, crop_size=256)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)


In [21]:
import os
import numpy as np

npy_dir = r"C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed"

# ✅ Track unique channel counts
channel_counts = set()

for file in os.listdir(npy_dir):
    if file.endswith(".npy"):
        eeg_data = np.load(os.path.join(npy_dir, file))
        channel_counts.add(eeg_data.shape[0])  # Get number of channels

print(f"✅ Unique channel counts in dataset: {channel_counts}")


✅ Unique channel counts in dataset: {32, 33, 34, 36, 41, 18, 20, 21, 30, 31}


In [22]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# ✅ Fixed number of channels for standardization
fixed_channels = 33  
crop_size = 256  

# ✅ Custom Dataset with Channel Standardization
class EEGDataset(Dataset):
    def __init__(self, npy_dir, crop_size=256, fixed_channels=33):
        self.npy_files = [os.path.join(npy_dir, f) for f in os.listdir(npy_dir) if f.endswith('.npy')]
        self.crop_size = crop_size
        self.fixed_channels = fixed_channels

    def __len__(self):
        return len(self.npy_files)

    def __getitem__(self, idx):
        # ✅ Load EEG segment
        eeg_data = np.load(self.npy_files[idx])
        num_channels, total_samples = eeg_data.shape

        # ✅ Standardize Channels (Zero-pad or Trim)
        if num_channels < self.fixed_channels:
            pad = np.zeros((self.fixed_channels - num_channels, total_samples))
            eeg_data = np.vstack((eeg_data, pad))  # Zero-pad missing channels
        elif num_channels > self.fixed_channels:
            eeg_data = eeg_data[:self.fixed_channels, :]  # Trim extra channels

        # ✅ Random Crop
        start_idx = np.random.randint(0, max(1, total_samples - self.crop_size + 1))
        eeg_cropped = eeg_data[:, start_idx:start_idx + self.crop_size]

        return torch.tensor(eeg_cropped, dtype=torch.float32)

# ✅ Create DataLoader
npy_dir = r"C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed"
train_dataset = EEGDataset(npy_dir, crop_size=256, fixed_channels=33)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)

print(f"✅ Dataset Loaded: {len(train_dataset)} samples")


✅ Dataset Loaded: 438 samples


In [23]:
eeg_1 = next(iter(train_loader))
print(f"✅ EEG Batch Shape: {eeg_1.shape}")  # Expected: (batch_size, 33, 256)


✅ EEG Batch Shape: torch.Size([8, 33, 256])


In [24]:
pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import torch
import torch.nn as nn

class TSTCC(nn.Module):
    def __init__(self, input_channels=33, seq_len=256, feature_dim=128):
        super(TSTCC, self).__init__()

        # ✅ CNN Encoder (Spatial Feature Extraction)
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=7, stride=2, padding=3)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv1d(256, feature_dim, kernel_size=3, stride=2, padding=1)

        # ✅ Transformer Encoder (Temporal Features)
        encoder_layer = nn.TransformerEncoderLayer(d_model=feature_dim, nhead=4)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        # ✅ Projection Head for Contrastive Learning
        self.projection_head = nn.Sequential(
            nn.Linear(feature_dim, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, feature_dim)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)  # Output: (batch, feature_dim, time)

        x = x.permute(2, 0, 1)  # Convert to (time, batch, features) for Transformer
        x = self.transformer(x)  # Temporal encoding
        x = x.mean(dim=0)  # Global pooling

        z = self.projection_head(x)  # Project to contrastive space
        return z

# ✅ Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TSTCC().to(device)

print(f"✅ Model initialized on {device}")


✅ Model initialized on cuda


c:\Users\alsaf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [26]:
import torch.nn.functional as F

class NTXentLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(NTXentLoss, self).__init__()
        self.temperature = temperature

    def forward(self, z1, z2):
        z1 = F.normalize(z1, dim=1)
        z2 = F.normalize(z2, dim=1)

        logits = torch.matmul(z1, z2.T) / self.temperature
        labels = torch.arange(z1.shape[0]).to(z1.device)

        loss = F.cross_entropy(logits, labels)
        return loss


In [27]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# ✅ Fixed number of channels
fixed_channels = 33  
crop_size = 256  

# ✅ Augmentation Function
def augment_eeg(eeg_data, crop_size=256, noise_level=0.05):
    """Apply random cropping & noise addition to EEG data."""
    total_samples = eeg_data.shape[1]

    # ✅ If EEG is shorter than crop size, use full length
    if total_samples < crop_size:
        crop_size = total_samples  

    # ✅ Random Crop
    start_idx = np.random.randint(0, max(1, total_samples - crop_size + 1))
    eeg_cropped = eeg_data[:, start_idx:start_idx + crop_size]

    # ✅ Add Gaussian Noise
    noise = noise_level * np.random.randn(*eeg_cropped.shape)
    eeg_noisy = eeg_cropped + noise

    return eeg_noisy

# ✅ Custom Dataset with Two Augmentations for TS-TCC
class EEGDataset(Dataset):
    def __init__(self, npy_dir, crop_size=256, fixed_channels=33):
        self.npy_files = [os.path.join(npy_dir, f) for f in os.listdir(npy_dir) if f.endswith('.npy')]
        self.crop_size = crop_size
        self.fixed_channels = fixed_channels

    def __len__(self):
        return len(self.npy_files)

    def __getitem__(self, idx):
        # ✅ Load EEG segment
        eeg_data = np.load(self.npy_files[idx])
        num_channels, total_samples = eeg_data.shape

        # ✅ Standardize Channels (Zero-pad or Trim)
        if num_channels < self.fixed_channels:
            pad = np.zeros((self.fixed_channels - num_channels, total_samples))
            eeg_data = np.vstack((eeg_data, pad))  
        elif num_channels > self.fixed_channels:
            eeg_data = eeg_data[:self.fixed_channels, :]  

        # ✅ Generate Two Augmented Views
        eeg_1 = augment_eeg(eeg_data, crop_size=self.crop_size)
        eeg_2 = augment_eeg(eeg_data, crop_size=self.crop_size)

        return torch.tensor(eeg_1, dtype=torch.float32), torch.tensor(eeg_2, dtype=torch.float32)

# ✅ Create DataLoader
npy_dir = r"C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed"
train_dataset = EEGDataset(npy_dir, crop_size=256, fixed_channels=33)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)

print(f"✅ Dataset Loaded: {len(train_dataset)} samples")


✅ Dataset Loaded: 438 samples


In [28]:
eeg_1, eeg_2 = next(iter(train_loader))

print(f"✅ EEG Batch 1 Shape: {eeg_1.shape}")  # Expected: (batch_size, 33, 256)
print(f"✅ EEG Batch 2 Shape: {eeg_2.shape}")  # Expected: (batch_size, 33, 256)


✅ EEG Batch 1 Shape: torch.Size([8, 33, 256])
✅ EEG Batch 2 Shape: torch.Size([8, 33, 256])


In [ ]:
import torch.optim as optim
import numpy as np

# ✅ Optimizer & Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
contrastive_loss = NTXentLoss().to(device)

# ✅ Training Loop with Early Stopping
def train_tstcc(model, train_loader, optimizer, epochs=100, patience=10):
    model.train()
    best_loss = np.inf  # Track best loss
    patience_counter = 0  # Counter for early stopping

    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (eeg_1, eeg_2) in enumerate(train_loader):
            eeg_1, eeg_2 = eeg_1.to(device), eeg_2.to(device)

            optimizer.zero_grad()
            z1 = model(eeg_1)
            z2 = model(eeg_2)

            loss = contrastive_loss(z1, z2)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if (batch_idx + 1) % 5 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        # ✅ Check Early Stopping Condition
        avg_loss = total_loss / len(train_loader)
        print(f"✅ Completed Epoch [{epoch+1}/{epochs}] | Avg Loss: {avg_loss:.4f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0  # Reset patience
            torch.save(model.state_dict(), "tstcc_best.pth")  # Save best model
            print("✅ Model Improved & Saved!")
        else:
            patience_counter += 1
            print(f"⏳ No Improvement | Patience: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("🛑 Early Stopping Triggered!")
            break

# ✅ Train with Early Stopping
train_tstcc(model, train_loader, optimizer, epochs=100, patience=10)


Epoch [1/100], Batch [5/55], Loss: 2.0794
Epoch [1/100], Batch [10/55], Loss: 2.0794
Epoch [1/100], Batch [15/55], Loss: 2.0789
Epoch [1/100], Batch [20/55], Loss: 2.0795
Epoch [1/100], Batch [25/55], Loss: 2.0793
Epoch [1/100], Batch [30/55], Loss: 2.0805
Epoch [1/100], Batch [35/55], Loss: 2.0787
Epoch [1/100], Batch [40/55], Loss: 2.0759
Epoch [1/100], Batch [45/55], Loss: 2.0617
Epoch [1/100], Batch [50/55], Loss: 2.0599
Epoch [1/100], Batch [55/55], Loss: 1.7836
✅ Completed Epoch [1/100] | Avg Loss: 2.0730
✅ Model Improved & Saved!
Epoch [2/100], Batch [5/55], Loss: 2.0794
Epoch [2/100], Batch [10/55], Loss: 2.0794
Epoch [2/100], Batch [15/55], Loss: 2.0794
Epoch [2/100], Batch [20/55], Loss: 2.0794
Epoch [2/100], Batch [25/55], Loss: 2.0794
Epoch [2/100], Batch [30/55], Loss: 2.0794
Epoch [2/100], Batch [35/55], Loss: 2.0794
Epoch [2/100], Batch [40/55], Loss: 2.0794
Epoch [2/100], Batch [45/55], Loss: 2.0795
Epoch [2/100], Batch [50/55], Loss: 2.0795
Epoch [2/100], Batch [55/55]

In [30]:
import torch
import torch.nn as nn

class TSTCC_Enhanced(nn.Module):
    def __init__(self, input_channels=33, seq_len=256, feature_dim=128):
        super(TSTCC_Enhanced, self).__init__()

        # ✅ Improved CNN Encoder (Deeper with BN & Dropout)
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm1d(256)
        self.conv4 = nn.Conv1d(256, feature_dim, kernel_size=3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm1d(feature_dim)
        self.dropout = nn.Dropout(0.2)

        # ✅ Deeper Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=feature_dim, nhead=8, dim_feedforward=512)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=4)

        # ✅ Projection Head for Contrastive Learning
        self.projection_head = nn.Sequential(
            nn.Linear(feature_dim, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, feature_dim)
        )

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.dropout(torch.relu(self.bn4(self.conv4(x))))  # Regularization

        x = x.permute(2, 0, 1)  # (time, batch, features) for Transformer
        x = self.transformer(x)  # Temporal encoding
        x = x.mean(dim=0)  # Global pooling

        z = self.projection_head(x)  # Contrastive Learning Projection
        return z

# ✅ Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TSTCC_Enhanced().to(device)

print(f"✅ Enhanced Model Initialized on {device}")


✅ Enhanced Model Initialized on cuda


In [31]:
import numpy as np
import random

# ✅ New Augmentation Function
def augment_eeg(eeg_data, crop_size=256, noise_level=0.05, freq_masking=True):
    total_samples = eeg_data.shape[1]

    # ✅ Random Crop
    start_idx = np.random.randint(0, max(1, total_samples - crop_size + 1))
    eeg_cropped = eeg_data[:, start_idx:start_idx + crop_size]

    # ✅ Add Gaussian Noise
    noise = noise_level * np.random.randn(*eeg_cropped.shape)
    eeg_noisy = eeg_cropped + noise

    # ✅ Frequency Masking (Randomly Zero-Out Frequencies)
    if freq_masking:
        num_masks = random.randint(1, 5)  # Apply 1-5 frequency masks
        for _ in range(num_masks):
            f = random.randint(0, eeg_cropped.shape[1] - 1)
            eeg_noisy[:, f:f+random.randint(2, 5)] = 0  

    return eeg_noisy

print("✅ Advanced Augmentation Ready!")


✅ Advanced Augmentation Ready!


In [32]:
import torch.optim as optim
import numpy as np

# ✅ Improved Optimizer & Learning Rate Scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)  # Smooth decay

contrastive_loss = NTXentLoss().to(device)

# ✅ Enhanced Training Loop with Early Stopping & LR Scheduling
def train_tstcc(model, train_loader, optimizer, scheduler, epochs=100, patience=10):
    model.train()
    best_loss = np.inf
    patience_counter = 0

    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (eeg_1, eeg_2) in enumerate(train_loader):
            eeg_1, eeg_2 = eeg_1.to(device), eeg_2.to(device)

            optimizer.zero_grad()
            z1 = model(eeg_1)
            z2 = model(eeg_2)

            loss = contrastive_loss(z1, z2)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if (batch_idx + 1) % 5 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        avg_loss = total_loss / len(train_loader)
        scheduler.step()  # Adjust learning rate dynamically

        print(f"✅ Completed Epoch [{epoch+1}/{epochs}] | Avg Loss: {avg_loss:.4f}")

        # ✅ Early Stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
            torch.save(model.state_dict(), "tstcc_best_enhanced.pth")
            print("✅ Model Improved & Saved!")
        else:
            patience_counter += 1
            print(f"⏳ No Improvement | Patience: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("🛑 Early Stopping Triggered!")
            break

# ✅ Train with Enhanced Model
train_tstcc(model, train_loader, optimizer, scheduler, epochs=100, patience=10)


Epoch [1/100], Batch [5/55], Loss: 1.2345
Epoch [1/100], Batch [10/55], Loss: 1.1400
Epoch [1/100], Batch [15/55], Loss: 1.1826
Epoch [1/100], Batch [20/55], Loss: 1.5844
Epoch [1/100], Batch [25/55], Loss: 1.1539
Epoch [1/100], Batch [30/55], Loss: 1.3055
Epoch [1/100], Batch [35/55], Loss: 1.5132
Epoch [1/100], Batch [40/55], Loss: 1.3023
Epoch [1/100], Batch [45/55], Loss: 1.5943
Epoch [1/100], Batch [50/55], Loss: 1.2131
Epoch [1/100], Batch [55/55], Loss: 0.9473
✅ Completed Epoch [1/100] | Avg Loss: 1.4246
✅ Model Improved & Saved!
Epoch [2/100], Batch [5/55], Loss: 1.1420
Epoch [2/100], Batch [10/55], Loss: 1.6677
Epoch [2/100], Batch [15/55], Loss: 1.0540
Epoch [2/100], Batch [20/55], Loss: 1.4271
Epoch [2/100], Batch [25/55], Loss: 1.3155
Epoch [2/100], Batch [30/55], Loss: 1.5663
Epoch [2/100], Batch [35/55], Loss: 1.1610
Epoch [2/100], Batch [40/55], Loss: 1.4512
Epoch [2/100], Batch [45/55], Loss: 1.7742
Epoch [2/100], Batch [50/55], Loss: 1.1500
Epoch [2/100], Batch [55/55]

In [33]:
import torch
import torch.nn as nn

class TSTCC_Optimized(nn.Module):
    def __init__(self, input_channels=33, seq_len=256, feature_dim=512):
        super(TSTCC_Optimized, self).__init__()

        # ✅ Deeper CNN Encoder
        self.conv1 = nn.Conv1d(input_channels, 128, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=5, stride=2, padding=2)
        self.bn2 = nn.BatchNorm1d(256)
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm1d(512)
        self.conv4 = nn.Conv1d(512, 512, kernel_size=3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(0.3)

        # ✅ Stronger Transformer (6 Layers)
        encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, dim_feedforward=1024)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=6)

        # ✅ Projection Head for Contrastive Learning
        self.projection_head = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 128)  # Reduced Dim for Better Generalization
        )

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.dropout(torch.relu(self.bn4(self.conv4(x))))  # Regularization

        x = x.permute(2, 0, 1)  # (time, batch, features) for Transformer
        x = self.transformer(x)  # Temporal encoding
        x = x.mean(dim=0)  # Global pooling

        z = self.projection_head(x)  # Contrastive Learning Projection
        return z

# ✅ Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TSTCC_Optimized().to(device)

print(f"✅ Optimized Model Initialized on {device}")


✅ Optimized Model Initialized on cuda


In [34]:
import numpy as np
import random

# ✅ Optimized Augmentation Function
def augment_eeg(eeg_data, crop_size=256, noise_level=0.02, freq_masking=True):
    total_samples = eeg_data.shape[1]

    # ✅ Random Crop
    start_idx = np.random.randint(0, max(1, total_samples - crop_size + 1))
    eeg_cropped = eeg_data[:, start_idx:start_idx + crop_size]

    # ✅ Time Shift (Simulates Variability)
    shift = random.randint(-10, 10)  # Shift by up to ±10 samples
    eeg_shifted = np.roll(eeg_cropped, shift, axis=1)

    # ✅ Add Gaussian Noise
    noise = noise_level * np.random.randn(*eeg_shifted.shape)
    eeg_noisy = eeg_shifted + noise

    # ✅ Frequency Masking (More Randomness)
    if freq_masking:
        num_masks = random.randint(1, 3)  # Apply 1-3 frequency masks
        for _ in range(num_masks):
            f = random.randint(0, eeg_noisy.shape[1] - 1)
            eeg_noisy[:, f:f+random.randint(3, 7)] = 0  

    return eeg_noisy

print("✅ Optimized Augmentation Ready!")


✅ Optimized Augmentation Ready!


In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# ✅ Optimized TS-TCC Model
class TSTCC_Optimized(nn.Module):
    def __init__(self, input_channels=33, seq_len=256, feature_dim=512):
        super(TSTCC_Optimized, self).__init__()

        self.conv1 = nn.Conv1d(input_channels, 128, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=5, stride=2, padding=2)
        self.bn2 = nn.BatchNorm1d(256)
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm1d(512)
        self.conv4 = nn.Conv1d(512, 512, kernel_size=3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(0.4)  # Increased dropout for better generalization

        encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, dim_feedforward=1024)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=6)

        self.projection_head = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 128)  # Lower dimensional embedding
        )

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.dropout(torch.relu(self.bn4(self.conv4(x))))

        x = x.permute(2, 0, 1)
        x = self.transformer(x)
        x = x.mean(dim=0)

        z = self.projection_head(x)
        return z

# ✅ Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TSTCC_Optimized().to(device)

print(f"✅ Optimized Model Initialized on {device}")


✅ Optimized Model Initialized on cuda


In [36]:
import random

def augment_eeg(eeg_data, crop_size=256, noise_level=0.02, freq_masking=True):
    total_samples = eeg_data.shape[1]

    # ✅ Better Cropping: Ensure different views
    start_idx_1 = np.random.randint(0, max(1, total_samples - crop_size + 1))
    start_idx_2 = np.random.randint(0, max(1, total_samples - crop_size + 1))

    eeg_1 = eeg_data[:, start_idx_1:start_idx_1 + crop_size]
    eeg_2 = eeg_data[:, start_idx_2:start_idx_2 + crop_size]

    # ✅ Slight amplitude scaling
    scale_factor = np.random.uniform(0.9, 1.1)
    eeg_1 *= scale_factor
    eeg_2 *= scale_factor

    # ✅ Add Gaussian Noise
    eeg_1 += noise_level * np.random.randn(*eeg_1.shape)
    eeg_2 += noise_level * np.random.randn(*eeg_2.shape)

    return eeg_1, eeg_2

print("✅ Optimized Augmentation Ready!")


✅ Optimized Augmentation Ready!


In [37]:
eeg_1, eeg_2 = eeg_1.to(device), eeg_2.to(device)


In [38]:
eeg_1, eeg_2 = eeg_1[:, 0].to(device), eeg_2[:, 1].to(device)  


In [39]:
for batch_idx, (eeg_1, eeg_2) in enumerate(train_loader):  # ✅ Fix Here
    eeg_1, eeg_2 = eeg_1.to(device), eeg_2.to(device)


C:\Users\alsaf\AppData\Local\Temp\ipykernel_204\3678052013.py:55: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  return torch.tensor(eeg_1, dtype=torch.float32), torch.tensor(eeg_2, dtype=torch.float32)


In [44]:
def train_tstcc(model, train_loader, optimizer, scheduler, epochs=100, patience=15):
    model.train()
    best_loss = np.inf
    patience_counter = 0

    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (eeg_1, eeg_2) in enumerate(train_loader):
            eeg_1, eeg_2 = eeg_1.squeeze(1).to(device), eeg_2.squeeze(1).to(device)  # ✅ Fix Shape

            optimizer.zero_grad()
            z1 = model(eeg_1)  # ✅ Now works
            z2 = model(eeg_2)

            loss = contrastive_loss(z1, z2)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if (batch_idx + 1) % 5 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        avg_loss = total_loss / len(train_loader)
        scheduler.step()

        print(f"✅ Completed Epoch [{epoch+1}/{epochs}] | Avg Loss: {avg_loss:.4f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
            torch.save(model.state_dict(), "tstcc_optimized.pth")
            print("✅ Model Improved & Saved!")
        else:
            patience_counter += 1
            print(f"⏳ No Improvement | Patience: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("🛑 Early Stopping Triggered!")
            break


BYOL

In [89]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import random

# ✅ Paths
npy_dir = r"C:\Users\alsaf\Downloads\Capstone2\EEG_Segments_Preprocessed"

# ✅ Standardize EEG shape
TARGET_CHANNELS = 33  # Most common channel count
CROP_SIZE = 5120  # Fixed length for all EEG samples

class EEGDataset(Dataset):
    def __init__(self, npy_dir):
        self.file_list = [os.path.join(npy_dir, f) for f in os.listdir(npy_dir) if f.endswith(".npy")]

    def standardize_eeg(self, eeg):
        """ Ensure EEG has fixed shape: (TARGET_CHANNELS, CROP_SIZE) """
        channels, timepoints = eeg.shape

        # ✅ Fix Channels (Pad or Trim)
        if channels < TARGET_CHANNELS:
            pad = np.zeros((TARGET_CHANNELS - channels, timepoints))
            eeg = np.vstack([eeg, pad])  # Pad missing channels
        elif channels > TARGET_CHANNELS:
            eeg = eeg[:TARGET_CHANNELS, :]  # Trim extra channels

        # ✅ Fix Time Length (Crop or Pad)
        if timepoints < CROP_SIZE:
            pad = np.zeros((TARGET_CHANNELS, CROP_SIZE - timepoints))
            eeg = np.hstack([eeg, pad])  # Pad missing timepoints
        elif timepoints > CROP_SIZE:
            start = random.randint(0, timepoints - CROP_SIZE)
            eeg = eeg[:, start:start + CROP_SIZE]  # Random crop

        return eeg

    def augment(self, eeg):
        """ Apply EEG-specific augmentations """
        eeg = eeg.copy()  # Ensure no modifications to the original data

        if random.random() > 0.5:
            eeg = np.flip(eeg, axis=1).copy()  # ✅ Copy after flipping to remove negative strides

        if random.random() > 0.5:
            noise = np.random.normal(0, 0.05, eeg.shape)  # Add small noise
            eeg = eeg + noise

        return eeg

    def __getitem__(self, idx):
        eeg_data = np.load(self.file_list[idx])  # Load raw EEG
        eeg_data = self.standardize_eeg(eeg_data)  # ✅ Ensure fixed shape

        # Create two augmented views for BYOL
        eeg_1 = self.augment(eeg_data)
        eeg_2 = self.augment(eeg_data)

        return torch.tensor(eeg_1, dtype=torch.float32), torch.tensor(eeg_2, dtype=torch.float32)

    def __len__(self):
        return len(self.file_list)

# ✅ Create DataLoader
train_dataset = EEGDataset(npy_dir)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)

# ✅ Check a batch
eeg_1, eeg_2 = next(iter(train_loader))
print(f"✅ EEG Batch Shape: {eeg_1.shape}")  # Expected: (batch_size, 33, 5120)


✅ EEG Batch Shape: torch.Size([8, 33, 5120])


In [90]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# ✅ Define the BYOL Model
class BYOL(nn.Module):
    def __init__(self, input_channels=33, hidden_dim=128, projection_dim=512):
        super(BYOL, self).__init__()

        # Feature Extractor
        self.encoder = nn.Sequential(
            nn.Conv1d(input_channels, hidden_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)  # Global Average Pooling
        )

        # Projection Head
        self.projector = nn.Sequential(
            nn.Linear(hidden_dim, projection_dim),
            nn.BatchNorm1d(projection_dim),
            nn.ReLU(),
            nn.Linear(projection_dim, projection_dim)
        )

        # Predictor Head
        self.predictor = nn.Sequential(
            nn.Linear(projection_dim, projection_dim // 2),
            nn.BatchNorm1d(projection_dim // 2),
            nn.ReLU(),
            nn.Linear(projection_dim // 2, projection_dim)
        )

    def forward(self, x1, x2):
        z1 = self.encoder(x1).squeeze(-1)
        z2 = self.encoder(x2).squeeze(-1)

        p1 = self.predictor(self.projector(z1))
        p2 = self.predictor(self.projector(z2))

        return p1, p2, z1, z2

# ✅ Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BYOL(input_channels=33).to(device)


In [91]:
# ✅ Define the Negative Cosine Similarity Loss for BYOL
def byol_loss(p1, p2, z1, z2):
    """ Negative cosine similarity loss """
    p1, p2, z1, z2 = map(lambda t: F.normalize(t, dim=-1), [p1, p2, z1, z2])  # Normalize embeddings

    loss = (2 - 2 * (p1 * z2).sum(dim=-1).mean()) + (2 - 2 * (p2 * z1).sum(dim=-1).mean())  # Contrastive loss
    return loss


In [93]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ✅ Define the BYOL Model
class BYOL(nn.Module):
    def __init__(self, input_channels=33, hidden_dim=128, projection_dim=128):  # ✅ Projection Dim = 128
        super(BYOL, self).__init__()

        # Feature Extractor
        self.encoder = nn.Sequential(
            nn.Conv1d(input_channels, hidden_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)  # Global Average Pooling
        )

        # Projection Head
        self.projector = nn.Sequential(
            nn.Linear(hidden_dim, projection_dim),
            nn.BatchNorm1d(projection_dim),
            nn.ReLU(),
            nn.Linear(projection_dim, projection_dim)  # ✅ Output size fixed to 128
        )

        # Predictor Head (Same size as projection now!)
        self.predictor = nn.Sequential(
            nn.Linear(projection_dim, projection_dim // 2),
            nn.BatchNorm1d(projection_dim // 2),
            nn.ReLU(),
            nn.Linear(projection_dim // 2, projection_dim)  # ✅ Output fixed to 128
        )

    def forward(self, x1, x2):
        """ Forward pass for BYOL: Extract, project & predict """
        z1 = self.encoder(x1).squeeze(-1)  # Extract features
        z2 = self.encoder(x2).squeeze(-1)

        z1 = self.projector(z1)  # Projection
        z2 = self.projector(z2)

        p1 = self.predictor(z1)  # Prediction
        p2 = self.predictor(z2)

        return p1, p2, z1, z2

# ✅ Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BYOL(input_channels=33).to(device)


In [94]:
# ✅ Define the Negative Cosine Similarity Loss for BYOL
def byol_loss(p1, p2, z1, z2):
    """ Negative cosine similarity loss """
    p1, p2, z1, z2 = map(lambda t: F.normalize(t, dim=-1), [p1, p2, z1, z2])  # Normalize embeddings

    # ✅ Debugging - Print Shapes
    print(f"Shapes - p1: {p1.shape}, p2: {p2.shape}, z1: {z1.shape}, z2: {z2.shape}")

    loss = (2 - 2 * (p1 * z2).sum(dim=-1).mean()) + (2 - 2 * (p2 * z1).sum(dim=-1).mean())  # Contrastive loss
    return loss


In [95]:
# ✅ Optimizer & Scheduler
import torch.optim as optim

optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# ✅ Training Loop with Early Stopping
def train_byol(model, train_loader, optimizer, scheduler, epochs=100, patience=15):
    model.train()
    best_loss = float("inf")
    patience_counter = 0

    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (eeg_1, eeg_2) in enumerate(train_loader):
            eeg_1, eeg_2 = eeg_1.to(device), eeg_2.to(device)

            optimizer.zero_grad()
            p1, p2, z1, z2 = model(eeg_1, eeg_2)

            loss = byol_loss(p1, p2, z1, z2)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if (batch_idx + 1) % 5 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        avg_loss = total_loss / len(train_loader)
        scheduler.step()

        print(f"✅ Completed Epoch [{epoch+1}/{epochs}] | Avg Loss: {avg_loss:.4f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
            torch.save(model.state_dict(), "byol_best.pth")
            print("✅ Model Improved & Saved!")
        else:
            patience_counter += 1
            print(f"⏳ No Improvement | Patience: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("🛑 Early Stopping Triggered!")
            break

# ✅ Train the Model
train_byol(model, train_loader, optimizer, scheduler, epochs=100, patience=15)


Shapes - p1: torch.Size([8, 128]), p2: torch.Size([8, 128]), z1: torch.Size([8, 128]), z2: torch.Size([8, 128])
Shapes - p1: torch.Size([8, 128]), p2: torch.Size([8, 128]), z1: torch.Size([8, 128]), z2: torch.Size([8, 128])
Shapes - p1: torch.Size([8, 128]), p2: torch.Size([8, 128]), z1: torch.Size([8, 128]), z2: torch.Size([8, 128])
Shapes - p1: torch.Size([8, 128]), p2: torch.Size([8, 128]), z1: torch.Size([8, 128]), z2: torch.Size([8, 128])
Shapes - p1: torch.Size([8, 128]), p2: torch.Size([8, 128]), z1: torch.Size([8, 128]), z2: torch.Size([8, 128])
Epoch [1/100], Batch [5/55], Loss: 3.2948
Shapes - p1: torch.Size([8, 128]), p2: torch.Size([8, 128]), z1: torch.Size([8, 128]), z2: torch.Size([8, 128])
Shapes - p1: torch.Size([8, 128]), p2: torch.Size([8, 128]), z1: torch.Size([8, 128]), z2: torch.Size([8, 128])
Shapes - p1: torch.Size([8, 128]), p2: torch.Size([8, 128]), z1: torch.Size([8, 128]), z2: torch.Size([8, 128])
Shapes - p1: torch.Size([8, 128]), p2: torch.Size([8, 128]), z

In [96]:
# ✅ Save Model Weights
torch.save(model.state_dict(), "byol_best.pth")
print("✅ Model weights saved successfully!")


✅ Model weights saved successfully!


In [98]:
# ✅ Ensure Model Definition Matches Training
input_channels = 33  # Set this to match your EEG dataset
hidden_dim = 128
projection_dim = 128

# ✅ Recreate the Model
model = BYOL(input_channels=input_channels, hidden_dim=hidden_dim, projection_dim=projection_dim)

# ✅ Load Trained Weights
model.load_state_dict(torch.load("byol_best.pth"))

# ✅ Move to Device & Set to Evaluation Mode
model.to(device)
model.eval()

print("✅ Model weights loaded successfully!")


✅ Model weights loaded successfully!


C:\Users\alsaf\AppData\Local\Temp\ipykernel_204\1282376678.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("byol_best.pth"))
